# 🥋 Lekcja 27: Gradient Checkpointing (Handel Czasem za Pamięć)

Podczas `forward()`, PyTorch domyślnie **zapisuje w pamięci** wszystkie wyniki pośrednie (aktywacje) każdej warstwy. Są one niezbędne do policzenia gradientów w `backward()`.
Jeśli masz 100 warstw, trzymasz 100 wielkich tensorów w VRAM.

**Idea Checkpointingu:**
1.  Nie zapisuj wyników pośrednich (np. warstw 10-90).
2.  Podczas `backward()`, gdy potrzebujesz tych wyników... **uruchom ten kawałek sieci jeszcze raz (Forward Re-computation)**.

**Wynik:**
*   Zużycie pamięci spada drastycznie (często o 50-70%).
*   Czas treningu rośnie o około 20-30% (bo liczysz forward dwa razy).

In [1]:
import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Urządzenie: {DEVICE}")

# Funkcja pomocnicza do mierzenia pamięci (Działa tylko na CUDA)
def print_memory(step_name):
    if torch.cuda.is_available():
        # Czekamy aż GPU skończy robotę
        torch.cuda.synchronize()
        allocated = torch.cuda.memory_allocated() / 1024**2 # MB
        print(f"[{step_name}] Zajęte VRAM: {allocated:.2f} MB")
    else:
        print(f"[{step_name}] (Brak GPU do pomiaru VRAM)")

Urządzenie: cuda


## Duży Model (Symulacja)

Stworzymy sieć z wielu ciężkich warstw liniowych, żeby zapchać pamięć.

In [2]:
# Ciężki blok (dużo obliczeń i duży wynik pośredni)
class HeavyBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2000, 2000)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.linear(x))

# Sieć złożona z 5 bloków
class BigNet(nn.Module):
    def __init__(self, use_checkpointing=False):
        super().__init__()
        self.use_checkpointing = use_checkpointing
        self.blocks = nn.ModuleList([HeavyBlock() for _ in range(10)]) # 10 bloków

    def forward(self, x):
        for block in self.blocks:
            if self.use_checkpointing:
                # MAGIA: Zamiast block(x), robimy checkpoint(block, x)
                # To mówi: "Nie zapisuj wyniku tego bloku. Odtwórz go w backwardzie."
                # Wymaga: dummy_arg (use_reentrant=False w nowszych wersjach jest bezpieczniejsze)
                x = checkpoint(block, x, use_reentrant=False)
            else:
                x = block(x)
        return x

print("Modele zdefiniowane.")

Modele zdefiniowane.


## Test 1: Standard (Pamięciożerny)

Uruchomimy model normalnie. Zobaczysz, jak pamięć rośnie, bo PyTorch musi trzymać wynik każdego z 10 bloków.

In [3]:
# Czyścimy pamięć
torch.cuda.empty_cache()
print_memory("Start")

model_std = BigNet(use_checkpointing=False).to(DEVICE)
input_data = torch.randn(128, 2000, requires_grad=True).to(DEVICE) # Spory batch

print_memory("Model załadowany")

# Forward
output = model_std(input_data)
print_memory("Po Forward (Standard)")

# Backward
loss = output.sum()
loss.backward()
print_memory("Po Backward")

# Sprzątanie
del model_std, input_data, output, loss
torch.cuda.empty_cache()

[Start] Zajęte VRAM: 0.00 MB
[Model załadowany] Zajęte VRAM: 161.05 MB
[Po Forward (Standard)] Zajęte VRAM: 179.95 MB
[Po Backward] Zajęte VRAM: 339.36 MB


## Test 2: Gradient Checkpointing (Oszczędny)

Teraz włączamy flagę.
W VRAM powinniśmy widzieć znacznie mniejsze zużycie po kroku `Forward`.
Dlaczego? Bo zamiast trzymać 10 dużych tensorów aktywacji, trzymamy tylko wejście i wyjście, a resztę zapomnieliśmy (odtworzymy na żądanie).

In [4]:
torch.cuda.empty_cache()
print_memory("Start Checkpoint")

model_ckpt = BigNet(use_checkpointing=True).to(DEVICE)
input_data = torch.randn(128, 2000, requires_grad=True).to(DEVICE)

print_memory("Model załadowany")

# Forward (Tu powinna być oszczędność!)
output = model_ckpt(input_data)
print_memory("Po Forward (Checkpointing)")

# Backward (Tu będzie wolniej, bo liczy forward jeszcze raz)
loss = output.sum()
loss.backward()
print_memory("Po Backward")

# Sprzątanie
del model_ckpt, input_data, output, loss
torch.cuda.empty_cache()

[Start Checkpoint] Zajęte VRAM: 17.25 MB
[Model załadowany] Zajęte VRAM: 178.30 MB
[Po Forward (Checkpointing)] Zajęte VRAM: 188.07 MB
[Po Backward] Zajęte VRAM: 340.36 MB


## 🥋 Black Belt Summary

1.  **Wynik:** Powinieneś widzieć, że "Po Forward (Standard)" zajmuje np. 200MB, a "Po Forward (Checkpointing)" np. 50MB. (Liczby zależą od karty).
2.  **Kiedy używać?**
    *   Trenujesz **LLM** (GPT, Llama) lub wielkie **ViT**.
    *   Dostajesz błąd `CUDA Out of Memory`.
    *   Chcesz zwiększyć Batch Size.
3.  **Gotowce:** W bibliotece `transformers` (HuggingFace) włącza się to jedną flagą: `model.gradient_checkpointing_enable()`. Pod spodem dzieje się dokładnie to, co napisaliśmy wyżej.